# Análisis de Base de Datos Meteorológica

Este notebook examina la estructura y contenido de la base de datos `meteorology_data.db`.

In [1]:
import sqlite3
import pandas as pd
import os

# Ruta a la base de datos
db_path = 'data/meteorology_data.db'

# Conexión a la base de datos
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print(f"Conectado a: {db_path}")

Conectado a: data/meteorology_data.db


In [2]:
# Obtener lista de tablas
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

print("Tablas en la base de datos:")
for table in tables:
    print(f"- {table[0]}")

Tablas en la base de datos:
- Magnitude
- data_source
- MaxValue
- MinValue
- MeanValue
- maxvalue_mapping
- minvalue_mapping
- meanvalue_mapping


In [3]:
# Examinar estructura de las tablas principales
print("=== ESQUEMA DE TABLAS ===\n")

for table in tables:
    table_name = table[0]
    print(f"Tabla: {table_name}")
    schema = cursor.execute(f"PRAGMA table_info({table_name});").fetchall()
    for col in schema:
        print(f"  {col[1]} ({col[2]})")
    print()

=== ESQUEMA DE TABLAS ===

Tabla: Magnitude
  Id (INTEGER)
  Name (VARCHAR(200))
  Unit (VARCHAR(50))
  Short (VARCHAR(50))
  GaugeExp (VARCHAR(500))

Tabla: data_source
  id (INTEGER)
  name (VARCHAR(200))
  description (VARCHAR(500))

Tabla: MaxValue
  Id (VARCHAR(36))
  Value (FLOAT)
  CurrentTime (DATETIME)
  MagnitudeId (INTEGER)

Tabla: MinValue
  Id (VARCHAR(36))
  Value (FLOAT)
  CurrentTime (DATETIME)
  MagnitudeId (INTEGER)

Tabla: MeanValue
  Id (VARCHAR(36))
  Value (FLOAT)
  StartTime (DATETIME)
  EndTime (DATETIME)
  MagnitudeId (INTEGER)

Tabla: maxvalue_mapping
  id (VARCHAR(36))
  source_num (INTEGER)
  original_id (INTEGER)
  target_id (VARCHAR(36))

Tabla: minvalue_mapping
  id (VARCHAR(36))
  source_num (INTEGER)
  original_id (INTEGER)
  target_id (VARCHAR(36))

Tabla: meanvalue_mapping
  id (VARCHAR(36))
  source_num (INTEGER)
  original_id (INTEGER)
  target_id (VARCHAR(36))



In [4]:
# Contenido de tabla Magnitude
print("=== CONTENIDO TABLA MAGNITUDE ===")
magnitude_df = pd.read_sql_query("SELECT * FROM Magnitude", conn)
print(f"Total magnitudes: {len(magnitude_df)}")
print(magnitude_df.to_string(index=False))

=== CONTENIDO TABLA MAGNITUDE ===
Total magnitudes: 16
 Id                                   Name Unit Short GaugeExp
  1                   Velocidad del Viento  m/s    ff     None
  2                   Dirección del Viento    °    dd     None
  3                   Temperatura del Aire   °C     T     None
  4              Humedad Relativa del Aire    %    HR     None
  5                    Presión Atmosférica  hPa     P     None
  6                 Radiación Solar Global W/m²     S     None
  7                Precipitación Acumulada   mm     R     None
  8          Temperatura de Punto de Rocío   °C    Td     None
  9        Presión al nivel de la Estación  hPa    P0     None
 10         Presión al nivel medio del Mar  hPa    P1     None
 11               Presión de Vapor de Agua  hPa     e     None
 12 Presión de Saturación de Vapor de Agua  hPa    es     None
 13            Temperatura de Enfriamiento   °C    WC     None
 14                        Índice de Calor   °C    IC     None


In [5]:
# Contenido de tabla data_source
print("=== CONTENIDO TABLA DATA_SOURCE ===")
source_df = pd.read_sql_query("SELECT * FROM data_source", conn)
print(f"Total fuentes: {len(source_df)}")
print(source_df.to_string(index=False))

=== CONTENIDO TABLA DATA_SOURCE ===
Total fuentes: 5
 id     name        description
  1 Source 2 Data from source 2
  2 Source 5 Data from source 5
  3 Source 1 Data from source 1
  4 Source 3 Data from source 3
  5 Source 4 Data from source 4


In [6]:
# Conteo de registros en tablas de valores
print("=== CONTEO DE REGISTROS ===")

tables_to_count = ['MeanValue', 'MaxValue', 'MinValue']

for table in tables_to_count:
    count = cursor.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    print(f"{table}: {count:,} registros")

=== CONTEO DE REGISTROS ===
MeanValue: 8,766,005 registros
MaxValue: 12,616,702 registros
MinValue: 6,256,561 registros


In [10]:
# Muestra de datos de valores promedio
print("=== MUESTRA DE VALORES PROMEDIO ===")
mean_sample = pd.read_sql_query("""
    SELECT mv.*, m.Name as MagnitudeName, m.Unit
    FROM MeanValue mv
    JOIN Magnitude m ON mv.MagnitudeId = m.Id
    LIMIT 30
""", conn)

print(mean_sample.to_string(index=False))

=== MUESTRA DE VALORES PROMEDIO ===
                                  Id  Value                  StartTime                    EndTime  MagnitudeId             MagnitudeName Unit
9f591d8e-60f6-41f4-b619-fd047d555267    3.7 2022-01-01 00:08:00.000000 2022-01-01 00:09:00.000000            1      Velocidad del Viento  m/s
086e5fe2-b565-45c5-a4c3-cde180feaa9a  161.0 2022-01-01 00:08:00.000000 2022-01-01 00:09:00.000000            2      Dirección del Viento    °
8904b7d1-6207-405d-8bc5-6161557c393f   24.1 2022-01-01 00:08:00.000000 2022-01-01 00:09:00.000000            3      Temperatura del Aire   °C
b541b4c7-1873-4821-8e1b-28d0435b9868   89.5 2022-01-01 00:08:00.000000 2022-01-01 00:09:00.000000            4 Humedad Relativa del Aire    %
b799647d-bfb4-4ab0-9b81-67d1ec382da3 1017.2 2022-01-01 00:08:00.000000 2022-01-01 00:09:00.000000            5       Presión Atmosférica  hPa
b282ac85-4707-49c5-93a1-6902e72dfc90   -1.6 2022-01-01 00:08:00.000000 2022-01-01 00:09:00.000000            6  

In [8]:
# Rango de fechas en los datos
print("=== RANGO DE FECHAS ===")

# Para MeanValue
date_range_mean = cursor.execute("""
    SELECT 
        MIN(StartTime) as min_start,
        MAX(EndTime) as max_end
    FROM MeanValue
""").fetchone()

print(f"MeanValue - Desde: {date_range_mean[0]} Hasta: {date_range_mean[1]}")

# Para MaxValue
date_range_max = cursor.execute("""
    SELECT 
        MIN(CurrentTime) as min_time,
        MAX(CurrentTime) as max_time
    FROM MaxValue
""").fetchone()

print(f"MaxValue - Desde: {date_range_max[0]} Hasta: {date_range_max[1]}")

=== RANGO DE FECHAS ===
MeanValue - Desde: 2022-01-01 00:08:00.000000 Hasta: 2025-11-01 09:29:00.000000
MaxValue - Desde: 2022-01-01 00:08:05.000000 Hasta: 2025-10-31 09:19:03.000000


In [ ]:
# Estadísticas por magnitud
print("=== ESTADÍSTICAS POR MAGNITUD ===")

stats_by_magnitude = pd.read_sql_query("""
    SELECT 
        m.Name,
        m.Unit,
        COUNT(mv.Id) as mean_count,
        COUNT(mv.Id) as max_count,
        COUNT(mv.Id) as min_count
    FROM Magnitude m
    LEFT JOIN MeanValue mv ON m.Id = mv.MagnitudeId
    LEFT JOIN MaxValue xv ON m.Id = xv.MagnitudeId
    LEFT JOIN MinValue nv ON m.Id = nv.MagnitudeId
    GROUP BY m.Id, m.Name, m.Unit
    ORDER BY m.Id
""", conn)

print(stats_by_magnitude.to_string(index=False))

In [ ]:
# Crear tabla pivote: días como filas, magnitudes como columnas
print("=== CREANDO TABLA DIARIA DE MAGNITUDES ===")

# Extraer fecha de StartTime y agrupar por día y magnitud
daily_data = pd.read_sql_query("""
    SELECT 
        DATE(StartTime) as date,
        MagnitudeId,
        AVG(Value) as daily_mean,
        -- Usar mediana para valores más robustos
        (
            SELECT AVG(Value) 
            FROM (
                SELECT Value 
                FROM MeanValue mv2 
                WHERE DATE(mv2.StartTime) = DATE(mv1.StartTime) 
                AND mv2.MagnitudeId = mv1.MagnitudeId
                ORDER BY Value 
                LIMIT 2 - (SELECT COUNT(*) FROM MeanValue mv3 
                          WHERE DATE(mv3.StartTime) = DATE(mv1.StartTime) 
                          AND mv3.MagnitudeId = mv1.MagnitudeId) % 2
                OFFSET (SELECT COUNT(*) - 1 FROM MeanValue mv4 
                        WHERE DATE(mv4.StartTime) = DATE(mv1.StartTime) 
                        AND mv4.MagnitudeId = mv1.MagnitudeId) / 2
            )
        ) as daily_median
    FROM MeanValue mv1
    WHERE MagnitudeId IN (1,2,3,4,5,6,7)
    GROUP BY DATE(StartTime), MagnitudeId
    ORDER BY date, MagnitudeId
""", conn)

print(f"Total registros diarios: {len(daily_data)}")
print("Primeros registros:")
print(daily_data.head(10))

In [11]:
# Crear tabla pivote: días como filas, magnitudes como columnas
print("=== CREANDO TABLA DIARIA DE MAGNITUDES ===")

# Primero obtener los datos básicos por día
daily_data = pd.read_sql_query("""
    SELECT 
        DATE(StartTime) as date,
        MagnitudeId,
        AVG(Value) as daily_mean,
        COUNT(*) as count_values
    FROM MeanValue
    WHERE MagnitudeId IN (1,2,3,4,5,6,7)
    GROUP BY DATE(StartTime), MagnitudeId
    ORDER BY date, MagnitudeId
""", conn)

print(f"Total registros diarios: {len(daily_data)}")
print("Primeros registros:")
print(daily_data.head(10))

=== CREANDO TABLA DIARIA DE MAGNITUDES ===
Total registros diarios: 9786
Primeros registros:
         date  MagnitudeId   daily_mean  count_values
0  2022-01-01            1     5.263542           288
1  2022-01-01            2   118.638889           288
2  2022-01-01            3    25.143056           288
3  2022-01-01            4    82.927778           288
4  2022-01-01            5  1016.496875           288
5  2022-01-01            6   195.332986           288
6  2022-01-01            7     0.000000           288
7  2022-01-02            1     4.513265           294
8  2022-01-02            2   134.418367           294
9  2022-01-02            3    24.760204           294


In [12]:
# Calcular medianas usando Python (más eficiente)
print("=== CALCULANDO MEDIANAS ===")

# Obtener todos los datos para calcular medianas en Python
all_daily_values = pd.read_sql_query("""
    SELECT 
        DATE(StartTime) as date,
        MagnitudeId,
        Value
    FROM MeanValue
    WHERE MagnitudeId IN (1,2,3,4,5,6,7)
    ORDER BY date, MagnitudeId, Value
""", conn)

# Calcular medianas por día y magnitud
daily_medians = all_daily_values.groupby(['date', 'MagnitudeId'])['Value'].median().reset_index()
daily_medians.columns = ['date', 'MagnitudeId', 'daily_median']

# Combinar con las medias
daily_data = daily_data.merge(daily_medians, on=['date', 'MagnitudeId'], how='left')

print("Datos combinados con medianas:")
print(daily_data.head(10))

=== CALCULANDO MEDIANAS ===
Datos combinados con medianas:
         date  MagnitudeId   daily_mean  count_values  daily_median
0  2022-01-01            1     5.263542           288          5.20
1  2022-01-01            2   118.638889           288        127.00
2  2022-01-01            3    25.143056           288         24.80
3  2022-01-01            4    82.927778           288         85.40
4  2022-01-01            5  1016.496875           288       1016.55
5  2022-01-01            6   195.332986           288         -2.05
6  2022-01-01            7     0.000000           288          0.00
7  2022-01-02            1     4.513265           294          4.10
8  2022-01-02            2   134.418367           294        134.00
9  2022-01-02            3    24.760204           294         24.40


In [13]:
# Crear tabla pivote final: días como filas, magnitudes como columnas (mean y median)
print("=== CREANDO TABLA PIVOTE FINAL ===")

# Obtener nombres de magnitudes para columnas
magnitudes = pd.read_sql_query("""
    SELECT Id, Name, Short 
    FROM Magnitude 
    WHERE Id IN (1,2,3,4,5,6,7)
    ORDER BY Id
""", conn)

print("Magnitudes disponibles:")
print(magnitudes.to_string(index=False))

# Crear tabla pivote para medias
pivot_mean = daily_data.pivot(
    index='date', 
    columns='MagnitudeId', 
    values='daily_mean'
)

# Crear tabla pivote para medianas
pivot_median = daily_data.pivot(
    index='date', 
    columns='MagnitudeId', 
    values='daily_median'
)

# Renombrar columnas con nombres descriptivos
column_names = {}
for _, row in magnitudes.iterrows():
    column_names[row['Id']] = f"{row['Short']}_{row['Name']}"

pivot_mean.columns = [f"{column_names.get(col, f'Mag_{col}')}_mean" for col in pivot_mean.columns]
pivot_median.columns = [f"{column_names.get(col, f'Mag_{col}')}_median" for col in pivot_median.columns]

# Combinar ambas tablas
pivot_final = pd.concat([pivot_mean, pivot_median], axis=1)

# Ordenar columnas: primero todas las medias, luego todas las medianas
mean_cols = [col for col in pivot_final.columns if col.endswith('_mean')]
median_cols = [col for col in pivot_final.columns if col.endswith('_median')]
pivot_final = pivot_final[mean_cols + median_cols]

print(f"\nTabla final: {pivot_final.shape[0]} días × {pivot_final.shape[1]} columnas")
print(f"Columnas: {len(mean_cols)} medias + {len(median_cols)} medianas")
print("\nPrimeras 5 filas:")
print(pivot_final.head())

=== CREANDO TABLA PIVOTE FINAL ===
Magnitudes disponibles:
 Id                      Name Short
  1      Velocidad del Viento    ff
  2      Dirección del Viento    dd
  3      Temperatura del Aire     T
  4 Humedad Relativa del Aire    HR
  5       Presión Atmosférica     P
  6    Radiación Solar Global     S
  7   Precipitación Acumulada     R

Tabla final: 1398 días × 14 columnas
Columnas: 7 medias + 7 medianas

Primeras 5 filas:
            ff_Velocidad del Viento_mean  dd_Dirección del Viento_mean  \
date                                                                     
2022-01-01                      5.263542                    118.638889   
2022-01-02                      4.513265                    134.418367   
2022-01-03                      3.056678                    245.097720   
2022-01-04                      3.287500                    270.145833   
2022-01-05                      1.943403                    196.013889   

            T_Temperatura del Aire_mean  HR_H

In [15]:
pivot_final.to_csv('data/daily_meteorological_data.csv')

In [16]:
# Cerrar conexión
conn.close()
print("Conexión cerrada.")

Conexión cerrada.
